In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import csv
from cleverhans.attacks import FastGradientMethod
from io import BytesIO
import IPython.display
import numpy as np
import pandas as pd
from PIL import Image
from scipy.misc import imread
from scipy.misc import imsave
import tensorflow as tf
from tensorflow.contrib.slim.nets import inception

In [ ]:
slim = tf.contrib.slim


tf.flags.DEFINE_string(
    'master', '', 'The address of the TensorFlow master to use.')

tf.flags.DEFINE_string(
    'checkpoint_path', '', '../input/inception-v3/inception_v3.ckpt')

tf.flags.DEFINE_string(
    'input_dir', '', '../input/images')

tf.flags.DEFINE_string(
    'output_dir', '', '../input')

tf.flags.DEFINE_float(
    'max_epsilon', 16.0, 'Maximum size of adversarial perturbation.')

tf.flags.DEFINE_integer(
    'image_width', 299, 'Width of each input images.')

tf.flags.DEFINE_integer(
    'image_height', 299, 'Height of each input images.')

tf.flags.DEFINE_integer(
    'num_channels', 3, 'How many images process at one time.')

tf.flags.DEFINE_integer(
    'batch_size', 16, 'How many images process at one time.')

tf.flags.DEFINE_float(
    'noise_limit', 0.35, 'Maximum size of adversarial perturbation.')

tf.flags.DEFINE_float(
    'noise_l2_weight', 0.02, 'Maximum size of adversarial perturbation.')
tf.flags.DEFINE_integer(
    'num_classes', 1001, 'How many images process at one time.')  


FLAGS = tf.flags.FLAGS

In [ ]:
batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
resized_image = [1,288,288,3]

In [ ]:
def load_target_class(input_dir):
    with tf.gfile.Open(os.path.join(input_dir, './input/images.csv')) as f:
        next(f) # skip header
        return {row[0]+".png": int(row[6]) for row in csv.reader(f) if len(row) >= 7}

def save_images(images, filenames, output_dir):
    for i, filename in enumerate(filenames):
        with tf.gfile.Open(os.path.join(output_dir, filename), 'wb') as f:
            imsave(f, (images[i, :, :, :] + 1.0) * 0.5, format='png')

def load_images(input_dir, batch_shape):
    images = np.zeros(batch_shape)
    filenames = []
    idx = 0
    batch_size = batch_shape[0]
    for filepath in sorted(tf.gfile.Glob(os.path.join(input_dir, '*.png'))):
        with tf.gfile.Open(filepath, "rb") as f:
            images[idx, :, :, :] = imread(f, mode='RGB').astype(np.float)*2.0/255.0 - 1.0
        filenames.append(os.path.basename(filepath))
        idx += 1
        if idx == batch_size:
            yield filenames, images
            filenames = []
            images = np.zeros(batch_shape)
            idx = 0
    if idx > 0:
        yield filenames, images

In [ ]:
 with tf.Graph().as_default():
    # Add a placeholder variable for the target class-number.
    # This will be set to e.g. 300 for the 'bookcase' class.
    x_input = tf.placeholder(tf.float32, shape=batch_shape)

    with slim.arg_scope(inception.inception_v3_arg_scope()):
        y_logits, end_points = inception.inception_v3(
              x_input, num_classes=FLAGS.num_classes, is_training=False)

        target_class_input = tf.placeholder(tf.int32, shape=[FLAGS.batch_size])
            
        one_hot_target_class = tf.one_hot(target_class_input, FLAGS.num_classes)
        pl_cls_target = tf.placeholder(dtype=tf.int32)

        # Add a new loss-function. This is the cross-entropy.
        # See Tutorial #01 for an explanation of cross-entropy.
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_logits, labels=[pl_cls_target])

        # Get the gradient for the loss-function with regard to
        # the resized input image.
        gradient = tf.gradients(loss, resized_image)

In [ ]:
  eps = 2.0 * FLAGS.max_epsilon / 255.0
  alpha = 2.0 * FLAGS.iter_alpha / 255.0
  num_iter = FLAGS.num_iter
  batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
  num_classes = 1001

  tf.logging.set_verbosity(tf.logging.INFO)

  all_images_taget_class = load_target_class(FLAGS.input_dir)

  with tf.Graph().as_default():
    # Prepare graph
    x_input = tf.placeholder(tf.float32, shape=batch_shape)
    x_max = tf.clip_by_value(x_input + eps, -1.0, 1.0)
    x_min = tf.clip_by_value(x_input - eps, -1.0, 1.0)

    with slim.arg_scope(inception.inception_v3_arg_scope()):
      inception.inception_v3(
          x_input, num_classes=num_classes, is_training=False)

    x_adv = x_input
    target_class_input = tf.placeholder(tf.int32, shape=[FLAGS.batch_size])
    one_hot_target_class = tf.one_hot(target_class_input, num_classes)

    for _ in range(num_iter):
      with slim.arg_scope(inception.inception_v3_arg_scope()):
        logits, end_points = inception.inception_v3(
            x_adv, num_classes=num_classes, is_training=False, reuse=True)
      cross_entropy = tf.losses.softmax_cross_entropy(one_hot_target_class,
                                                      logits,
                                                      label_smoothing=0.1,
                                                      weights=1.0)
      cross_entropy += tf.losses.softmax_cross_entropy(one_hot_target_class,
                                                       end_points['AuxLogits'],
                                                       label_smoothing=0.1,
                                                       weights=0.4)
      x_next = x_adv - alpha * tf.sign(tf.gradients(cross_entropy, x_adv)[0])
      x_next = tf.clip_by_value(x_next, x_min, x_max)
      x_adv = x_next

    # Run computation
    saver = tf.train.Saver(slim.get_model_variables())
    session_creator = tf.train.ChiefSessionCreator(
        scaffold=tf.train.Scaffold(saver=saver),
        checkpoint_filename_with_path=FLAGS.checkpoint_path,
        master=FLAGS.master)


In [ ]:
def normalize_image(x):
    # Get the min and max values for all pixels in the input.
    x_min = x.min()
    x_max = x.max()

    # Normalize so all values are between 0.0 and 1.0
    x_norm = (x - x_min) / (x_max - x_min)

    return x_norm

In [ ]:
def adversary_noise(image_path, cls_target, noise_limit=3.0,
                         required_score=0.99, max_iterations=100):
    

    # Create a feed-dict with the image.
    
    feed_dict = model._create_feed_dict(image_path=image_path)
   
 

    # Use TensorFlow to calculate the predicted class-scores
    # (aka. probabilities) as well as the resized image.
    pred, image = session.run([y_pred, resized_image],
                              feed_dict=feed_dict)

    # Convert to one-dimensional array.
    pred = np.squeeze(pred)

    # Predicted class-number.
    cls_source = np.argmax(pred)

    # Score for the predicted class (aka. probability or confidence).
    score_source_org = pred.max()

    # Names for the source and target classes.
    name_source = model.name_lookup.cls_to_name(cls_source,
                                                only_first_name=True)
    name_target = model.name_lookup.cls_to_name(cls_target,
                                                only_first_name=True)

    # Initialize the noise to zero.
    noise = 0

    # Perform a number of optimization iterations to find
    # the noise that causes mis-classification of the input image.
    for i in range(max_iterations):
        print("Iteration:", i)

        # The noisy image is just the sum of the input image and noise.
        noisy_image = image + noise

        
        noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

        feed_dict = {model.tensor_name_resized_image: noisy_image,
                     pl_cls_target: cls_target}

        # Calculate the predicted class-scores as well as the gradient.
        pred, grad = session.run([y_pred, gradient],
                                 feed_dict=feed_dict)

        # Convert the predicted class-scores to a one-dim array.
        pred = np.squeeze(pred)

        # The scores (probabilities) for the source and target classes.
        score_source = pred[cls_source]
        score_target = pred[cls_target]

        # Squeeze the dimensionality for the gradient-array.
        grad = np.array(grad).squeeze()

        
        grad_absmax = np.abs(grad).max()
        
        # If the gradient is very small then use a lower limit,
        # because we will use it as a divisor.
        if grad_absmax < 1e-10:
            grad_absmax = 1e-10

        step_size = 7 / grad_absmax

        

        # If the score for the target-class is not high enough.
        if score_target < required_score:
            # Update the image-noise by subtracting the gradient
            # scaled by the step-size.
            noise -= step_size * grad

            # Ensure the noise is within the desired range.
            # This avoids distorting the image too much.
            noise = np.clip(a=noise,
                            a_min=-noise_limit,
                            a_max=noise_limit)
        else:
            # Abort the optimization because the score is high enough.
            break

    return image.squeeze(), noisy_image.squeeze(), noise, \
           name_source, name_target, \
           score_source, score_source_org, score_target

In [ ]:
def target_attack(image_path, cls_target,
                      noise_limit, required_score):
   

    # Find the adversarial noise.
    image, noisy_image, noise, \
    name_source, name_target, \
    score_source, score_source_org, score_target = \
        adversary_noise(image_path=image_path,
                             cls_target=cls_target,
                             noise_limit=noise_limit,
                             required_score=required_score)


In [ ]:
if __name__ == '__main__':
    with tf.train.MonitoredSession(session_creator=session_creator) as sess:
          for filenames, images in load_images(FLAGS.input_dir, batch_shape):
            target_class_for_batch = (
            [all_images_taget_class[n] for n in filenames]
            + [0] * (FLAGS.batch_size - len(filenames)))

            #sess.run(tf.global_variables_initializer())
            #sess.run(tf.variables_initializer([x_noise]))
            adv_images = sess.run(target_attack(image_path, cls_target,
                      noise_limit, required_score))
            #sess.run(x_noise_clip)
            save_images(adv_images, filenames, FLAGS.output_dir)